## 读取MovieLens 数据

In [1]:
import pandas
training_set = pandas.read_csv('../ml-100k/u1.base', sep = '\t', names = ['userid', 'itemid', 'rating', 'tm'])
training_set.head()

,userid,itemid,rating,tm
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [2]:
import pandas
test_set = pandas.read_csv('../ml-100k/u1.test', sep = '\t', names = ['userid', 'itemid', 'rating', 'tm'])
test_set.head()

,userid,itemid,rating,tm
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


## 计算电影与使用者数量

In [3]:
n_movies = max(max(training_set.itemid.tolist()), max(test_set.itemid.tolist()))
n_movies

1682

In [4]:
n_users= max(max(training_set.userid.tolist()), max(test_set.userid.tolist()))
n_users

943

## 建立训练数据集矩阵

In [5]:
import numpy as np
training_m = np.zeros((n_users, n_movies))
for rec in training_set.iterrows():
    training_m[rec[1].userid - 1 , rec[1].itemid - 1] = 1

In [6]:
training_m.shape

(943, 1682)

In [7]:
training_m

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

## 建立测试数据集矩阵

In [8]:
import numpy as np
test_m = np.zeros((n_users, n_movies))
for rec in test_set.iterrows():
    test_m[rec[1].userid - 1 , rec[1].itemid - 1] = 1

In [9]:
test_m.shape

(943, 1682)

In [10]:
test_m

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 建立Autoencoders

In [11]:
from keras.layers import Input, Dense
from keras.models import Model

encoding_dim = 50  
input_data = Input(shape=(n_users,))
encoded = Dense(encoding_dim, activation='softmax')(input_data)
decoded = Dense(n_users)(encoded)

autoencoder = Model(input_data, decoded)
autoencoder.compile(optimizer='adam', loss='mean_absolute_error')

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


## 训练Autoencoders

In [12]:
autoencoder.fit(training_m.T, training_m.T,
                epochs=100,
                batch_size=32,
                shuffle=True,
                validation_data=(test_m.T, test_m.T) )

Train on 1682 samples, validate on 1682 samples
Epoch 1/100
1682/1682 [==============================] - 0s 219us/step - loss: 0.0518 - val_loss: 0.0132
Epoch 2/100
1682/1682 [==============================] - 0s 131us/step - loss: 0.0511 - val_loss: 0.0131
Epoch 3/100
1682/1682 [==============================] - 0s 131us/step - loss: 0.0510 - val_loss: 0.0130
Epoch 4/100
1682/1682 [==============================] - 0s 130us/step - loss: 0.0508 - val_loss: 0.0130
Epoch 5/100
1682/1682 [==============================] - 0s 137us/step - loss: 0.0508 - val_loss: 0.0129
Epoch 6/100
1682/1682 [==============================] - 0s 138us/step - loss: 0.0507 - val_loss: 0.0129
Epoch 7/100
1682/1682 [==============================] - 0s 142us/step - loss: 0.0507 - val_loss: 0.0129
Epoch 8/100
1682/1682 [==============================] - 0s 139us/step - loss: 0.0507 - val_loss: 0.0129
Epoch 9/100
1682/1682 [==============================] - 0s 136us/step - loss: 0.0506 - val_loss: 0.0129
Epoch 1

Epoch 78/100
1682/1682 [==============================] - 0s 161us/step - loss: 0.0480 - val_loss: 0.0131
Epoch 79/100
1682/1682 [==============================] - 0s 155us/step - loss: 0.0480 - val_loss: 0.0131
Epoch 80/100
1682/1682 [==============================] - 0s 154us/step - loss: 0.0480 - val_loss: 0.0131
Epoch 81/100
1682/1682 [==============================] - 0s 156us/step - loss: 0.0479 - val_loss: 0.0131
Epoch 82/100
1682/1682 [==============================] - 0s 160us/step - loss: 0.0479 - val_loss: 0.0131
Epoch 83/100
1682/1682 [==============================] - 0s 162us/step - loss: 0.0479 - val_loss: 0.0131
Epoch 84/100
1682/1682 [==============================] - 0s 175us/step - loss: 0.0479 - val_loss: 0.0131
Epoch 85/100
1682/1682 [==============================] - 0s 163us/step - loss: 0.0479 - val_loss: 0.0131
Epoch 86/100
1682/1682 [==============================] - 0s 158us/step - loss: 0.0479 - val_loss: 0.0131
Epoch 87/100
1682/1682 [======================

## 建立 Encoder

In [16]:
encoder = Model(input_data, encoded)

In [17]:
encoded_data = encoder.predict(training_m.T)

In [18]:
encoded_data.shape

(1682, 50)

## 建立 Decoder

In [19]:
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [20]:
pred = decoder.predict(encoded_data)

In [22]:
pred.shape

(1682, 943)

## 检视原始与重建数据

In [26]:
#pred
(pred[:,0] > 0.001).astype(int)

array([0, 0, 0, ..., 0, 0, 0])

In [27]:
sum((pred[:,0] > 0.001).astype(int) == training_m.T[:,0]) / len(training_m.T[:,0])

0.9197384066587396

In [28]:
((pred > 0.001).astype(int) == training_m.T).sum() / (1682 * 943)

0.9412058058439241

In [29]:
pred[ training_m.T == 0] = 0
mse = np.mean(np.power(training_m.T.flatten() - pred.flatten(), 2))
mse

0.04462493029919939

## 套用到测试数据集

In [30]:
encoded_test_data = encoder.predict(test_m.T)

In [31]:
pred_test = decoder.predict(encoded_test_data)

In [32]:
((pred_test > 0.001).astype(int) == test_m.T).sum() / (1682 * 943)

0.9583557674484877

In [33]:
pred_test[ test_m.T == 0] = 0
mse = np.mean(np.power(test_m.T.flatten() - pred_test.flatten(), 2))
mse

0.012590241568206168